classification

- static
- determenistic
- fully observable
- discrete
- single agent
- general

# Automated planning

Is a way of finding plans (sequences of actions) that lead from an initial state to a goal state.
Here we look at classic planning
- General approach to finding solutions for state space search problems
- Classic = static, deterministic, fully observable
- Variants: Probalistic planning, planning under partial observability, online planning, etc.

Given
- A state space description in terms of suitable problem description language (planning formalism)

Required
- A plan, i.e. a solution for the described state space (sequence of actions from initial state to goal state)
- Or proof that no plan exists

Distinguish between
- Optimal planning: Guarantee that the returned plans are optimal, i.e., have minimal overall cost
- Suboptimal planning (satisficing): Suboptimal plans are allowed

# Four planning

A description language for state spaces (planning tasks) is called a planning formalism.

# STRIPS

- It is the most simple common planning formalism.
- State variables are binary (true or false)
- States $s$ (based on a given set of state variables $V$) can be represented in two equivalent ways
    - As assignments $s: V \rightarrow \{\mathbf{T}, \mathbf{F}\}$
    - As sets $s \subseteq V$ where $s$ encodes the set of state variables that are true in $s$.
- Goals and preconditions of actions are given as sets of variables that must be true (values of other variables do not matter)
- Effects of actions are given as sets of variables that are set to true and set to false, respectively

## Definition

A STRIPS planning task is a 4 tuple $\prod = \langle V, I, G, A \rangle$ with
- $V$: Finite set of state variables
- $I \subseteq V$: The initial state
- $G \subseteq V$: The set of goals
- $A$: Finite set of actions where for all actions $a \in A$, the following is defined:
    - $pre(a) \subseteq V$: The preconditions of $a$
    - $add(a) \subseteq V$: The add effects of $a$
    - $del(a) \subseteq V$: The delete effects of $a$
    - $cost(a) \in \mathbb{N}_0$: The cost of $a$

Given thr STRIPS planning task. Then $\prod$ induces the state space $\mathcal{S}(\prod) = \langle S, A, cost, T, s_I, S_G \rangle$:
- Set of states $S = 2^V$
- Actions: actions $A$ as defined in $\prod$
- Action costs: $cost$ as defined in $\prod$
- Transitions: $s \xrightarrow[]{a} s'$ for states $s,s' \in S$ and action $a \in A$ iff
    - $pre(a) \subseteq s$ (precondition satisfied)
    - $s' = (s \textbackslash del(a)) \cup add(a)$ (effects are applied)
- Initial state: $s_I = I$
- Goal state: $s \in S_G$ for state $s$ iff $G \subseteq s$ (goals reached)

# ADL

Like STRIPS, ADL uses propositional variables (true / false) as state variables. Preconditions of actions and goal are arbitrary logic formulas (action applicable / goal reached in states that satisfy the formula). In Addition to STRIPS effects, there are conditional effects: variable $v$ is only set to true / false if a given logical formula is true in the current state.

# SAS+

Very similair to STRIPS: state variables not necessarily binary, but with given finite domain. States are assignments to these variables. Preconditions and goals given as partial assignments. Effects are assignments to subset of variables.

# PDDL

Is standard language used in practice to describe planning tasks. Descriptions in predicate logic instead of propositional logic. Other features lke numeric variables and derived variables (axioms) for defining complex logical conditions (formulas that are automatically evaluated in every state and can, e.g. be used in preconditions). There exist defined PDDL fragments for STRIPS and ADL: many planners only support the STRIPS fragment.